# Esercitazione Guidata: Analisi di Dati con Python e Pandas

Benvenuti a questa esercitazione pratica! L'obiettivo di oggi è imparare a esplorare, pulire e visualizzare un dataset reale utilizzando le librerie Python che sono il pane quotidiano di ogni Data Analyst: **Pandas**, **NumPy** e **Matplotlib**.

Lavoreremo con il dataset `Dataset Abitudini Sportive`, che raccoglie informazioni sulle abitudini di allenamento di diverse persone. Impareremo a scoprire cosa si nasconde dietro i numeri e a rispondere a domande concrete basate sui dati.

In [ ]:
# Importiamo le librerie che ci servono
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Un piccolo comando per migliorare lo stile dei grafici
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Caricamento del Dataset

Per prima cosa, dobbiamo caricare i nostri dati. Assicurati di aver caricato il file `Dataset Abitudini Sportive.csv` nella sessione di Colab (puoi semplicemente trascinarlo nella cartella a sinistra).

In [ ]:
# Specifichiamo il nome del file
file_path = 'Dataset Abitudini Sportive.csv'

# Carichiamo il file CSV in un DataFrame di Pandas
df = pd.read_csv(file_path)

# Creiamo una copia del DataFrame originale che useremo più avanti
df_originale = df.copy()

## 2. Analisi Esplorativa Iniziale

Ora che i dati sono caricati, diamo una prima occhiata per capire con cosa abbiamo a che fare. È come aprire il cofano di una macchina per la prima volta: non capiremo subito tutto, ma ci faremo un'idea generale.

### `df.head()` - Le prime 5 righe
Il comando `head()` ci mostra un'anteprima del dataset, utilissima per capire al volo quali sono le colonne e che tipo di dati contengono.

In [ ]:
df.head()

### `df.info()` - Informazioni sul DataFrame
Il metodo `info()` è fondamentale. Ci fornisce una sintesi delle colonne, il numero di valori non nulli e il tipo di dato (Dtype) di ogni colonna. È il nostro primo strumento per scovare eventuali problemi, come dati mancanti o tipi di dato sbagliati.

In [ ]:
df.info()

### `df.describe()` - Statistiche di base
Con `describe()`, otteniamo un riassunto statistico per tutte le colonne numeriche: media, deviazione standard, minimo, massimo e i percentili. Questo ci aiuta a capire la distribuzione dei dati e a identificare possibili valori anomali (outlier).

In [ ]:
df.describe()

### Interpretazione delle Statistiche: Dove si nascondono i dati sporchi?

Osserviamo attentamente l'output di `describe()`. Questa tabella è una miniera d'oro per scovare problemi.
- **`count`**: Notiamo che alcune colonne hanno meno di 1000 valori. Questo conferma la presenza di dati mancanti che `info()` aveva già suggerito.
- **`mean` (media)**: Guardiamo la media di `ore_settimanali_allenamento`. Sembra molto bassa, quasi sospetta. Una media così bassa potrebbe essere "abbassata" da valori estremamente piccoli e anomali.
- **`min` (minimo)**: Ecco il campanello d'allarme più evidente! Colonne come `ore_settimanali_allenamento` e `attrezzatura_comprata_annualmente` mostrano un valore minimo di `-999`. È fisicamente impossibile allenarsi per un numero negativo di ore o spendere una cifra negativa. Questo è un chiaro segno di un valore usato per codificare "dato mancante" o "non applicabile".
- **`max` (massimo)**: I valori massimi sembrano plausibili, ma è sempre bene controllarli per vedere se ci sono valori esagerati (es. 200 ore di allenamento a settimana).

--- 
**Domanda 1:** Osservando la tabella `describe()`, quali altre colonne, oltre a `ore_settimanali_allenamento`, presentano un valore minimo anomalo di `-999`? Elencale.

## 3. Pulizia e Gestione dei Dati Mancanti

Abbiamo confermato che il nostro dataset è "sporco". Prima di poter fare analisi affidabili, dobbiamo pulirlo. Il primo passo è standardizzare tutti i valori mancanti in un formato che Pandas possa riconoscere: `np.nan` (Not a Number).

In [ ]:
# Copiamo il dataframe per sicurezza, così possiamo confrontare i risultati
df_pulito = df.copy()

# 1. Standardizziamo i valori mancanti
df_pulito.replace([-999, '***', ''], np.nan, inplace=True)

# 2. Correggiamo i tipi di dato
colonne_numeriche = ['ore_settimanali_allenamento', 'frequenza_allenamento', 'soddisfazione_allenamento', 'attrezzatura_comprata_annualmente', 'frequenza_infortuni', 'alimentazione_durante_allenamento', 'tempo_riposo_settimanale', 'Y']
for colonna in colonne_numeriche:
    df_pulito[colonna] = pd.to_numeric(df_pulito[colonna], errors='coerce')

# 3. Verifichiamo il risultato
print("Dati mancanti dopo la standardizzazione:")
df_pulito.isnull().sum()

### Strategie per Gestire i Dati Mancanti
Ora che tutti i dati mancanti sono `NaN`, cosa facciamo? Esistono diverse strategie, ognuna con i suoi pro e contro. Vediamone due.

#### Strategia 1: Eliminazione delle Righe (Poco consigliata)
La soluzione più semplice è eliminare ogni riga che contiene anche solo un valore mancante. Si usa il comando `dropna()`.

**Pro:** È facile e veloce.
**Contro:** Si rischia di perdere una grande quantità di dati! Se una riga ha 9 valori su 10 e ne manca solo uno, la eliminiamo comunque, sprecando informazioni preziose.

In [ ]:
# Vediamo quante righe rimarrebbero se usassimo dropna()
righe_originali = len(df_pulito)
righe_dopo_drop = len(df_pulito.dropna())

print(f"Righe originali: {righe_originali}")
print(f"Righe dopo aver eliminato i NaN: {righe_dopo_drop}")
print(f"Percentuale di dati persi: {((righe_originali - righe_dopo_drop) / righe_originali) * 100:.2f}%")

Come puoi vedere, perderemmo quasi un quarto del dataset! Per questo motivo, questa strategia è sconsigliata a meno che i dati mancanti non siano pochissimi.

#### Strategia 2: Imputazione (La scelta migliore qui)
L'imputazione consiste nel **sostituire** i valori mancanti con una stima plausibile. Le stime più comuni per le colonne numeriche sono la **media** o la **mediana** della colonna stessa.

- **Media:** Ottima se i dati hanno una distribuzione simmetrica (a campana) e non ci sono troppi outlier.
- **Mediana:** Molto più robusta in presenza di outlier o distribuzioni asimmetriche, perché rappresenta il valore centrale.

Sostituiamo i `NaN` nella colonna `ore_settimanali_allenamento` con la sua **mediana**.

In [ ]:
# 1. Calcoliamo la mediana
mediana_ore = df_pulito['ore_settimanali_allenamento'].median()
print(f"La mediana delle ore di allenamento è: {mediana_ore}")

# 2. Usiamo il metodo fillna() per sostituire i NaN
df_pulito['ore_settimanali_allenamento'].fillna(mediana_ore, inplace=True)

# 3. Verifichiamo che non ci siano più NaN in quella colonna
print("\nValori mancanti dopo l'imputazione:")
df_pulito.isnull().sum()

--- 
**Esercizio 1:** Ora tocca a te! La colonna `attrezzatura_comprata_annualmente` ha ancora dei valori mancanti. Completa il codice qui sotto per sostituire i `NaN` con la **media** di quella colonna.

In [ ]:
# Completa il codice
media_attrezzatura = df_pulito['attrezzatura_comprata_annualmente'].mean() # Calcola la media
df_pulito['attrezzatura_comprata_annualmente'].fillna(media_attrezzatura, inplace=True) # Sostituisci i NaN

# Codice di verifica (non modificarlo)
if df_pulito['attrezzatura_comprata_annualmente'].isnull().sum() == 0:
    print("Ottimo lavoro! I valori mancanti sono stati riempiti.")
else:
    print("Riprova, ci sono ancora dei valori mancanti.")

### Completamento della Pulizia

Ottimo lavoro! Abbiamo gestito due colonne, ma ne rimangono altre con dati mancanti. Un buon data analyst deve assicurarsi che il dataset sia **completamente pulito** prima di procedere. Applichiamo una strategia di imputazione a tutte le colonne rimanenti.

- Per le colonne **numeriche**, useremo la **mediana** (più robusta).
- Per le colonne **categoriche** (testuali), useremo la **moda**, ovvero il valore più frequente.

In [ ]:
# Selezioniamo le colonne rimanenti con NaN
colonne_con_nan = df_pulito.columns[df_pulito.isnull().any()].tolist()
print(f"Colonne ancora da pulire: {colonne_con_nan}")

# Usiamo un ciclo per applicare l'imputazione appropriata a ciascuna
for colonna in colonne_con_nan:
    if pd.api.types.is_numeric_dtype(df_pulito[colonna]):
        # Se la colonna è numerica, usiamo la mediana
        mediana = df_pulito[colonna].median()
        df_pulito[colonna].fillna(mediana, inplace=True)
        print(f"Imputati i valori in '{colonna}' (numerica) con la mediana ({mediana}).")
    else:
        # Se la colonna è categorica, usiamo la moda
        moda = df_pulito[colonna].mode()[0] # .mode() restituisce una Serie, prendiamo il primo elemento
        df_pulito[colonna].fillna(moda, inplace=True)
        print(f"Imputati i valori in '{colonna}' (categorica) con la moda ('{moda}').")

# Verifica finale
print("\nVerifica finale dei dati mancanti:")
df_pulito.isnull().sum()

Perfetto! Ora il nostro DataFrame `df_pulito` non ha più valori mancanti ed è pronto per essere analizzato e visualizzato in modo affidabile.

## 4. Visualizzazione dei Dati

I grafici sono il modo migliore per "vedere" i dati e scoprire pattern che i numeri da soli non mostrano. Creeremo alcuni grafici di base per esplorare le colonne più interessanti del nostro dataset pulito.

### Istogramma - Distribuzione delle ore di allenamento
Un istogramma è perfetto per visualizzare la distribuzione di una variabile numerica, come le ore di allenamento settimanali. Ci mostra quante persone rientrano in ciascun "intervallo" di ore.

In [ ]:
plt.figure(figsize=(10, 6))
df_pulito['ore_settimanali_allenamento'].hist(bins=15, edgecolor='black')
plt.title('Distribuzione delle Ore di Allenamento Settimanali')
plt.xlabel('Ore di Allenamento')
plt.ylabel('Numero di Persone')
plt.show()

### Grafico a Barre - Obiettivi di Allenamento
Un grafico a barre è l'ideale per le variabili categoriche. Lo useremo per contare quante persone hanno ciascun obiettivo di allenamento.

In [ ]:
plt.figure(figsize=(12, 7))
df_pulito['obiettivo_allenamento_label'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Frequenza degli Obiettivi di Allenamento')
plt.xlabel('Obiettivo')
plt.ylabel('Numero di Persone')
plt.xticks(rotation=45, ha='right')
plt.tight_layout() # Aggiusta il layout per non tagliare le etichette
plt.show()

--- 
**Esercizio 2:** Abbiamo visto come creare un grafico a barre per gli obiettivi. Ora crea tu un grafico a barre per la colonna `tempo_riposo_settimanale_label`. Ricordati di dare un titolo e nomi appropriati agli assi!

In [ ]:
# Scrivi qui il tuo codice per creare il grafico a barre


## 5. Formulazione e Verifica di Ipotesi

Questa è la parte più divertente: usare i dati per rispondere a delle domande! Formuliamo due semplici ipotesi e vediamo se i dati le confermano.

### Ipotesi 1: Chi punta alla perdita di peso compra meno attrezzatura di chi punta al miglioramento muscolare?

**Logica:** Vogliamo confrontare la media di `attrezzatura_comprata_annualmente` per due gruppi specifici. Per farlo, dobbiamo:
1.  Filtrare il DataFrame per selezionare solo le righe che ci interessano.
2.  Calcolare la media per ciascun gruppo.

In [ ]:
# Filtriamo i due gruppi
gruppo_perdita_peso = df_pulito[df_pulito['obiettivo_allenamento_label'] == 'Perdita peso']
gruppo_massa_muscolare = df_pulito[df_pulito['obiettivo_allenamento_label'] == 'Miglioramento muscolare']

# Calcoliamo la media per ciascun gruppo
media_peso = gruppo_perdita_peso['attrezzatura_comprata_annualmente'].mean()
media_massa = gruppo_massa_muscolare['attrezzatura_comprata_annualmente'].mean()

print(f"Media spesa per attrezzatura (Perdita peso): {media_peso:.2f} €")
print(f"Media spesa per attrezzatura (Miglioramento muscolare): {media_massa:.2f} €")

--- 
**Esercizio 3:** Abbiamo confrontato la media, ma forse la **mediana** è una metrica più robusta in questo caso. Modifica il codice qui sopra per calcolare e stampare la **mediana** della spesa per attrezzatura per i due gruppi. Cosa noti di diverso rispetto alla media?

In [ ]:
# Scrivi qui il codice modificato per calcolare la mediana


### Ipotesi 2: C'è una relazione tra la frequenza di allenamento e la soddisfazione?

**Logica:** Vogliamo vedere se, in media, le persone che si allenano più spesso sono anche più soddisfatte. Il metodo `groupby()` è perfetto per questo: raggruppa tutte le righe in base a una colonna (la frequenza) e ci permette di calcolare una statistica (la media della soddisfazione) per ogni gruppo.

In [ ]:
# Raggruppiamo per frequenza e calcoliamo la soddisfazione media
soddisfazione_per_frequenza = df_pulito.groupby('frequenza_allenamento')['soddisfazione_allenamento'].mean()

print("Soddisfazione media per frequenza di allenamento:")
print(soddisfazione_per_frequenza)

# --- SOLUZIONI ---
--- 
---

## Soluzioni degli Esercizi

### Risposta alla Domanda 1
Le colonne che presentano un valore minimo di -999 sono: `ore_settimanali_allenamento`, `soddisfazione_allenamento`, `attrezzatura_comprata_annualmente`, `frequenza_infortuni`, `alimentazione_durante_allenamento`, `tempo_riposo_settimanale` e `Y`.

### Soluzione Esercizio 1

In [ ]:
# Calcola la media
media_attrezzatura = df_pulito['attrezzatura_comprata_annualmente'].mean()
# Sostituisci i NaN
df_pulito['attrezzatura_comprata_annualmente'].fillna(media_attrezzatura, inplace=True)

print("Verifica per l'esercizio 1:")
if df_pulito['attrezzatura_comprata_annualmente'].isnull().sum() == 0:
    print("Corretto! I valori mancanti sono stati riempiti.")
else:
    print("Qualcosa è andato storto.")

### Soluzione Esercizio 2

In [ ]:
plt.figure(figsize=(10, 6))
df_pulito['tempo_riposo_settimanale_label'].value_counts().plot(kind='bar', color='lightgreen')
plt.title('Frequenza del Riposo Settimanale')
plt.xlabel('Giorni di Riposo')
plt.ylabel('Numero di Persone')
plt.xticks(rotation=0)
plt.show()

### Soluzione Esercizio 3

In [ ]:
# Calcoliamo la mediana per ciascun gruppo
mediana_peso = gruppo_perdita_peso['attrezzatura_comprata_annualmente'].median()
mediana_massa = gruppo_massa_muscolare['attrezzatura_comprata_annualmente'].median()

print(f"Mediana spesa per attrezzatura (Perdita peso): {mediana_peso:.2f} €")
print(f"Mediana spesa per attrezzatura (Miglioramento muscolare): {mediana_massa:.2f} €")